# Comparison with pandas

[pandas](https://pandas.pydata.org) is a popular Python library for data analysis and manipulation. 

GreenplumPython strives to provide a pandas-like interface so that people can get started with it quickly.

In [1]:
import pandas as pd
import greenplumpython as gp
gp

<module 'greenplumpython' from '/home/gpadmin/.local/lib/python3.9/site-packages/greenplumpython/__init__.py'>

However, due to the discrepency in the purposes they serve, that is,
- GreenplumPython is an interface to a remote database system, while
- pandas is a library for manipulating local in-memory data

there are still some important differences in their interface.

## Data Structure

The core data structure of GreenplumPython, `Table`, is fundamentally similar to `Dataframe` in pandas in that
- Data are organized into rows and columns;
- Columns can be of different types, and can be accessed by name;
- Rows are of the same type, and are iterable.

Next, we will see similarities and differences between them in detail with examples.

### Getting Access to the Structure

For example, suppose we have some information of students, including students' names and ages:

In [2]:
students = [("alice", 18), ("bob", 19), ("carol", 19)]
students

[('alice', 18), ('bob', 19), ('carol', 19)]

For analyzing them, we might want to create a pandas `DataFrame` as follows:

In [3]:
df = pd.DataFrame.from_records(students, columns=["name", "age"])
df

,name,age
0,alice,18
1,bob,19
2,carol,19


We can also create a `Table` in GreenplumPython from the same data in a very similar way:

In [4]:
db = gp.database(dbname="gpadmin")
t = gp.to_table(students, column_names=["name", "age"], db=db)
t

name,age
alice,18
bob,19
carol,19


But here is an important **difference**:
- a `Table` in GreenplumPython must be created in a database, while
- pandas does not have the concept of "database".

`Database` in GreenplumPython is like "directory" in file systems, which helps to avoid name conflict on persistence.

The reason behind this difference is that
- `Table`s in GreenplumPython can themselves be persisted, while
- `DataFrame`s in pandas live only in memory and need to be written to a file for persistence.

A `Table` can be saved persistently in the database system with

In [5]:
t.save_as("student", temp=True)

name,age
bob,19
carol,19
alice,18


This is similar to how a `DataFrame` is persisted as file

In [6]:
df.to_csv("/tmp/student.csv")

Saving a `Table` with `temp=True` in GreenplumPython is similar to saving a `DataFrame` into the `/tmp` directory
- `Table`s saved with `temp=True` will be dropped automatically by the database system, while
- `DataFrame`s saved in `/tmp` will be clean automatically by the operating system.

In order to get access to a previously persisted `Table` in a `Database`, like `student`, we can do

In [7]:
student = db.table("student")
student

name,age
bob,19
carol,19
alice,18


This is similar to loading a `DataFrame` from file in pandas, that is,

In [8]:
pd.read_csv("/tmp/student.csv")

,Unnamed: 0,name,age
0,0,alice,18
1,1,bob,19
2,2,carol,19


### Accessing Data in Rows and Columns

In both GreenplumPython `Table`s and pandas `DataFrame`s, rows can be iterated.

To access each row in a `Table` in GreenplumPython, simply iterate over the `Table`:

In [9]:
for row in t:
    print(row["name"], row["age"])

alice 18
bob 19
carol 19


This is similar to how rows in a pandas `DataFrame` can be accessed:

In [10]:
for row in df.iterrows():
    print(row[1]["name"], row[1]["age"])

alice 18
bob 19
carol 19


Similar to pandas, `Row` in GreenplumPython is `dict`-like. The value of each column can be accessed by name.

In conclusion, from the user's perspective, `Table` in GreenplumPython is very similar to `DataFrame` in pandas. We expect this would make it easier for whoever interested to get started using GreenplumPython.

## Data Selection

Data selection is probably the most fundamental set of operations on data.

In both pandas and GreenplumPython, data selection is done primarily with the `[]` operator.

### Selecting Columns

In both pandas and GreenplumPython, columns are accessed by name.

For example, to select a subset of columns, such as `name` and `age`, from the dataframe containing student info, in pandas we can do

In [11]:
df[["name", "age"]]

,name,age
0,alice,18
1,bob,19
2,carol,19


The result of the `[]` operator is a new `DataFrame`. In GreenplumPython, this is exactly the same:

In [12]:
student[["name", "age"]]

name,age
bob,19
carol,19
alice,18


The result is a new `Table` containing the selected columns.

### Accessing a Single Column

To refer to a single column, we can use the `[]` operator with the column name in both pandas and GreenplumPython.

For example, to access the names of the students in pandas, we can do:

In [13]:
df["name"]

0    alice
1      bob
2    carol
Name: name, dtype: object

While in GreenplumPython, the column can be refered to in the same way:

In [14]:
t["name"]

But you might notice the **difference** here:
- for pandas, using the `[]` operator gives us the data immediately if we refer to a column, while
- for GreenplumPython, it only gives a `Column` object. `Column` is supposed to be used for computation rather than for observing data.

The reasons behind this difference are
- Database systems behind GreenplumPython does not provide native one-dimensional data structure like `Series` in pandas.
- It is much more efficient to retrieve all columns needed in a `Table` at once than one at a time. We will see later how to add new columns to a `Table` so that they can be retrived all at once.

### Selecting Rows by Predicate

The `[]` operator can also be used to select a subset of rows, a.k.a filtering.

Say we want the infomation of student named "alice", with pandas we can do

In [15]:
df[lambda df: df["name"] == "alice"]

,name,age
0,alice,18


With GreenplumPython, we can do it in exactly the same way:

In [16]:
student[lambda t: t["name"] == "alice"]

name,age
alice,18


Here we see how a column in GreenplumPython, `t["name"]` in this case, is used for computation to form a more complex expression.

In this example, When the expression `t["name"] == "alice"` is evaluated, `t` will be bound to the "current" table, i.e. `student`.

GreenplumPython provides such a functional interface so that the user does not have to refer to the possibly long intermediate variable name like `student` again and again when the expression becomes complicated.

### Selecting Rows by Slice

We can get a quick glance of the data by selecting the first several rows. This can be achieved with `slice` in Python.

Like many built-in data structures in Python, such as `list` and `tuple`, `Table` in GreenplumPython supports slicing.

For example, if we want only the first two rows of the `Table` of students in GreenplumPython, we can do

In [17]:
student[:2]

name,age
alice,18
bob,19


In pandas, we can do exactly the same thing on a `DataFrame`:

In [18]:
df[:2]

,name,age
0,alice,18
1,bob,19


But you might notice the **difference**: When selecting rows,
- for pandas, rows in the output `DataFrame` preserves the same order as the input, while
- for GreenplumPython, the order of rows in `Table` might not be preserved.

The difference is due to the fact that database systems behind will not guarantee the order of rows unless otherwise specified.


## Data Transformation

Data transformation is about changing the data to a desired form.

Like pandas, GreenplumPython provides powerful building blocks to make transformation easier.

### Data Ordering

Having the data sorted in a desired order makes it convenient for many analytical tasks, such as statistics.

pandas supports sorting the data (a.k.a values) of a `DataFrame` by columns.

For example, we can sort in pandas the dataframe of student info by "age" and then "name", both in descending order with

In [19]:
df.sort_values(["age", "name"], ascending=[False, False])

,name,age
2,carol,19
1,bob,19
0,alice,18


In GreenplumPython, order of data can be defined with the `order_by()` method:

In [20]:
student.order_by("age", ascending=False).order_by("name", ascending=False)[:]

name,age
carol,19
bob,19
alice,18


There are some important **difference** compared with pandas:
- GreenplumPython does not provide something like `DataFrame.sort_index()` in pandas, because unlike `DataFrame`s in pandas, `Table` in GreenplumPython does not have an "index column".
- `Table.order_by()` supports chaining to specify multiple ordering columns. We think that compared to passing lists, method chaining is simpler because it is not required to remember the order of columns in the list. 

### Column Transformation

Column transformation is to transform one or more existing columns into a new one of the same length.

A new column may contains data resulting from whatever computation we want. 

Both GreeplumPython and pandas support transforming columns by adding a new column. Specifically, we need to
- define the transfomation as an expression, and then
- bind the expression to a new column of the source `Table` or `DataFrame` to form a new one.

We can use `assign()` method to add new columns. For example, suppose we would like to know the year of birth for each student in the previous example.

In pandas, we can add a new column named `year_of_birth` like this

In [21]:
import datetime 

this_year = datetime.date.today().year
df.assign(year_of_birth=lambda df: -df["age"] + this_year)

,name,age,year_of_birth
0,alice,18,2004
1,bob,19,2003
2,carol,19,2003


In GreenplymPython, we can do exactly the same:

In [22]:
student.assign(year_of_birth=lambda t: -t["age"] + this_year)

name,age,year_of_birth
alice,18,2004
bob,19,2003
carol,19,2003


The column-defining expression can be a function call, which can contain more complex computation.

For example, in order to hide the names of students to protect privacy, we can write a function transforming names to something not human-readable.

In [23]:
@gp.create_function
def hash_name(name: str) -> str:
    from hashlib import sha256
    
    return sha256(name.encode("utf-8")).hexdigest()

The `gp.create_function` decorator converts a Python function into a User-Defined Function (UDF) in database so that it can be applied to `Column`s.

With the function defined, we can then apply it to generate a new `Column`:

In [24]:
student.assign(name_=lambda t: hash_name(t["name"]))

name,age,name_
bob,19,81b637d8fcd2c6da6359e6963113a1170de795e4b725b84d1e0b4cfd9ec58ce9
carol,19,4c26d9074c27d89ede59270c0ac14b71e071b15239519f75474b2f3ba63481f5
alice,18,2bd806c97f0e00af1a1fc3328fa763a9269723c8db8fac4f93af71db186d6e90


After adding the new column, we can select the columns we care about into a new `Table` with the `[]` operator.

To be more concise, GreenplumPython and pandas support transforming columns directly into a new `Table` or `DataFrame` by `apply()`-ing the function.

In the previous example, using `apply()`, we can obtain the `Table` with the original names hidden: 

In [25]:
class Student:
    name: str
    age: int

def hide_name(name: str, age: int) -> Student:
    from hashlib import sha256
    
    return {"name": sha256(name.encode("utf-8")).hexdigest(), "age": age}

student.apply(lambda t: gp.create_function(hide_name)(t["name"], t["age"]), expand=True)

name,age
2bd806c97f0e00af1a1fc3328fa763a9269723c8db8fac4f93af71db186d6e90,18
81b637d8fcd2c6da6359e6963113a1170de795e4b725b84d1e0b4cfd9ec58ce9,19
4c26d9074c27d89ede59270c0ac14b71e071b15239519f75474b2f3ba63481f5,19


We can directly apply the same Python function without any modification to the `DataFrame` in pandas:

In [26]:
df.apply(lambda df: hide_name(df["name"], df["age"]), axis=1, result_type="expand")

,name,age
0,2bd806c97f0e00af1a1fc3328fa763a9269723c8db8fac...,18
1,81b637d8fcd2c6da6359e6963113a1170de795e4b725b8...,19
2,4c26d9074c27d89ede59270c0ac14b71e071b15239519f...,19


But there are still some important **differences** between the two cases:
- In pandas, what we apply to a `DataFrame` is a Python function, while in GreenplumPython, a Python function must be converted to a database function before being applied to a `Table`.
- pandas supports applying a function along different axes, while GreenplumPython only support applying a function to each row due to the limitation of the database system behind.

### Data Grouping

Like ordering, Grouping data based on distinct set of values of columns can also facilitate analytical tasks.

Both GreenplumPython and pandas support data grouping, in a very similar way.

Data grouping is often associated with aggregation to obtain statistics. For example, suppose we want to the number of students of different age.

In pandas, we can do

In [27]:
import numpy as np

df.groupby("age").apply(lambda df: np.count_nonzero(df["name"]))

age
18    1
19    2
dtype: int64

In GreenplumPython, what we need to do is

In [28]:
count = gp.aggregate_function("count")

student.group_by("age").apply(lambda t: count(t["name"]))

count,age
2,19
1,18


### Data Deduplication

Data duplication is supported in both GreenplumPython and pandas, also in a very similar way.

For example, suppose we want to draw a representative sample containing students for each distinct age,

With pandas, we can do

In [30]:
df.drop_duplicates("age")

,name,age
0,alice,18
1,bob,19


With GreenplumPython, what we need to do is

In [31]:
student.distinct_on("age")

name,age
alice,18
carol,19


Moreover, GreenplumPython also supports aggregation on only the distinct values.

Suppose we want to know the number of different ages of the students, we can do

In [32]:
student.apply(lambda t: count.distinct(t["age"]))

count
2
